In [127]:
import pandas as pd
import numpy as np
import shutil, os, re, sqlite3

<h2 style='color:blue'>O código adiante obtém os arquivos csv, por tipo (rat, rat_evetivo, rat_viatura e rat_produtividade) e os ordena por data</h2>

In [129]:
# dir = os.getcwd()
# rats_files_names = os.listdir(dir+'/files/RAT')
# rat_pattern = re.compile('REDS_RAT_2')
# rat_viatura_pattern = re.compile('RAT_VIATURA')
# rat_efetivo_pattern = re.compile('RAT_EFETIVO')
# rat_produtividade_pattern = re.compile('RAT_Produtividade')

# rats_files = list()
# rats_viatura_files = list()
# rats_efetivo_files = list()
# rats_produtividade_files = list()

# for file_name in rats_files_names:
#     is_rat = rat_pattern.search(file_name) != None
#     is_rat_viatura = rat_viatura_pattern.search(file_name) != None
#     is_rat_efetivo = rat_efetivo_pattern.search(file_name) != None
#     is_rat_produtividade = rat_produtividade_pattern.search(file_name) != None
#     if is_rat:
#         rats_files.append(file_name)
#     elif is_rat_viatura:
#         rats_viatura_files.append(file_name)
#     elif is_rat_efetivo:
#         rats_efetivo_files.append(file_name)
#     elif is_rat_produtividade:
#         rats_produtividade_files.append(file_name)
        
# rats_files.sort(key=lambda name: int(name[9:17]))
# rats_viatura_files.sort(key=lambda name: int(name[18:26]))
# rats_efetivo_files.sort(key=lambda name: int(name[18:26]))
# rats_produtividade_files.sort(key=lambda name: int(name[23:31]))

<h2 style="color:blue;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT</h2>
<h3 style="color:red; margin:10px">Importa somente os dados do 23º BPM</h3>

### 1) Lê o primeiro arquivo da lista e o passa para um DataFrame (df1)
### 2) Lê o segundo arquivo da lista e o passa para outro DataFrame (df2)
### 3) Concatena os dois DataFrames em um terceiro (df_acum)
### 4) Itera sobre os demais arquivos, lendo cada um deles, aplicando strip(), a função de filtro do 23º BPM e depois o adiciona ao DataFrame df_acum
### 5) Retira as duplicatas, pelo campo 'RAT.NUM_ATIVIDADE'
### 6) Insere os dados de df_acum no banco de dados gdo.db: (tabela tbl_rat)

In [131]:
# df_classif.drop_duplicates()
# df_classif_dup = df_classif_dup[df_classif_dup > 1]
df_classif = pd.read_csv('files/classificadores.csv')
df_classif.drop_duplicates(['MUNICIPIO', 'VALIDADOR_TIPO', 'VALIDADOR'], inplace=True)
df_classif.set_index(df_classif['MUNICIPIO'] + " " + df_classif['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR'], inplace=True)
df_classif
# df_classif.to_csv('files/classificadores.csv')

# # df_classif_new['MUN_VALTIPO_VAL'] = df_classif_new['MUNICIPIO'] + " " + df_classif_new['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR']
# df_classif_new['MUN_VALTIPO_VAL'].value_counts()

,Unnamed: 0,MUNICIPIO,VALIDADOR,VALIDADOR_TIPO,CIA_BASE,BAIRRO,TIPO,SETOR,SUBSETOR,COD_SETOR,OBSERVACAO
ARAUJOS LOGRADOURO MALAQUIAS,0,ARAUJOS,MALAQUIAS,LOGRADOURO,280ª CIA PM,ZONA RURAL,RURAL,ZONA RURAL,ZONA RURAL,60,ZONA RUAL 279ª CIA PM
CARMO DO CAJURU BAIRRO JARDIM ALVORADA,1,CARMO DO CAJURU,JARDIM ALVORADA,BAIRRO,142ª CIA PM,JARDIM ALVORADA,URBANO,PORTO VELHO,NaN,NaN,NaN
CARMO DO CAJURU BAIRRO CHACARAS BELA VISTA,2,CARMO DO CAJURU,CHACARAS BELA VISTA,BAIRRO,142ª CIA PM,CHACARAS BELA VISTA,RURAL,NITEROI,NaN,NaN,AO DOS ESTRELA OESTE CLUBE SEDE CAMPESTRE
CARMO DO CAJURU N_RAT 2018-009340806-001,3,CARMO DO CAJURU,2018-009340806-001,N_RAT,142ª CIA PM,CHACARAS BELA VISTA,RURAL,NITEROI,NaN,NaN,KM 109 CARMO DO CAJURU / BALANÇA
CONCEICAO DO PARA BAIRRO_NAO_CAD POVOADO DE CUNHA - ZONA RURAL DE CONCEI¿¿O DO PAR¿,4,CONCEICAO DO PARA,POVOADO DE CUNHA - ZONA RURAL DE CONCEI¿¿O DO ...,BAIRRO_NAO_CAD,280ª CIA PM,CUNHAS,RURAL,ZONA RURAL,ZONA RURAL,60,CORRESPONDE AO SETOR PERIFÉRICO DA UEOP


In [126]:
df_teste['D'] = np.select([
    df_teste['A'].isin(df_TESTE['A'])
],[
    5
], default=1)


str

In [122]:
df_teste = pd.DataFrame({
    'A':list('bacdef'),
    'B':list('dbxydg'),
    'C':list('mbxymg')
})
df_TESTE = pd.DataFrame({
    'A':list('aBCMEd'),
    'B':list('DBXAEO'),
    'C':list('XYZWPT')
})
pd.concat([df_teste, df_TESTE], axis=1)

,A,B,C,A,B,C
0,b,d,m,a,D,X
1,a,b,b,B,B,Y
2,c,x,x,C,X,Z
3,d,y,y,M,A,W
4,e,d,m,E,E,P
5,f,g,g,d,O,T


In [231]:
# df_classif = pd.read_csv('files/classificadores.csv')
# df_classif.drop_duplicates(['MUNICIPIO', 'VALIDADOR_TIPO', 'VALIDADOR'], inplace=True)
# df_classif.set_index(df_classif['MUNICIPIO'] + " " + df_classif['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR'], inplace=True)


# def filter_23(df):
#     return df[
#         df['MUNICIPIO'].isin([
#             'DIVINOPOLIS',
#             'ITAUNA',
#             'ITATIAIUCU',
#             'CARMO DO CAJURU',
#             'SAO GONCALO DO PARA',
#             'CLAUDIO'
#         ])
#     ]


# df1 = pd.read_csv('files/RAT/' + rats_files[0], error_bad_lines=False, sep='|')
# df1 = df1.applymap(lambda x: x.strip() if type(x) == str else x)
# df1 = filter_23(df1)

# df2 = pd.read_csv('files/RAT/' + rats_files[1], error_bad_lines=False, sep='|')
# df2 = df2.applymap(lambda x: x.strip() if type(x) == str else x)
# df2 = filter_23(df2)

# df_acum = pd.concat([df1, df2])

# for rat_file in rats_files[2:]:
#     df_aux = pd.read_csv('files/RAT/' + rat_file, error_bad_lines=False, sep='|')
#     df_aux = df_aux.applymap(lambda x: x.strip() if type(x) == str else x)
#     df_aux = filter_23(df_aux)
#     df_acum = pd.concat([df_acum, df_aux])

# df_acum.drop_duplicates(subset='RAT.NUM_ATIVIDADE', keep='last', inplace=True)

# df_acum.loc[:,'DTA_HRA_INICIO'] = df_acum['DTA_INICIO'] + " " + df_acum['HRA_INICIO']
# df_acum.loc[:,'DTA_HRA_INICIO_DT'] = pd.to_datetime( df_acum['DTA_HRA_INICIO'], format='%d/%m/%Y %H:%M', errors='coerce')

# df_acum.loc[:,'DTA_HRA_TERMINO'] = df_acum.loc[:,'DTA_TERMINO'] + " " + df_acum.loc[:,'HRA_TERMINO']
# df_acum.loc[:,'DTA_HRA_TERMINO_DT'] = pd.to_datetime( df_acum.loc[:,'DTA_HRA_TERMINO'], format='%d/%m/%Y %H:%M', errors='coerce')

# df_acum.loc[:,'MES'] = df_acum.loc[:,'DTA_HRA_INICIO_DT'].dt.month
# df_acum.loc[:,'ANO'] = df_acum.loc[:,'DTA_HRA_INICIO_DT'].dt.year

# df_acum.loc[:,'TEMPO_DT'] = df_acum['DTA_HRA_TERMINO_DT'] - df_acum['DTA_HRA_INICIO_DT']

# df_acum.loc[:,'TEMPO_INT'] = df_acum['TEMPO_DT'].dt.total_seconds() / 60

# cond139 = df_acum['NOM_UNIDADE_AREA'].str.contains('139 CIA')
# cond142 = df_acum['NOM_UNIDADE_AREA'].str.contains('142 CIA')
# cond53 = df_acum['NOM_UNIDADE_AREA'].str.contains('53 CIA')
# cond51 = (
#     ( df_acum['NOM_UNIDADE_AREA'].str.contains('51 CIA') ) |
#     ( df_acum['NOM_UNIDADE_AREA'].str.contains('63 BPM') ) |
#     ( df_acum['NOM_UNIDADE_AREA'].str.contains('DESAT5') )
# )

# conditions = [cond139, cond142, cond53, cond51]
# result = ['139 CIA', '142 CIA', '53 CIA', '51 CIA']

# df_acum.loc[:, 'CIA'] = np.select(conditions, result )

# df_acum.head()

def classifica_setor(row):
    mun = row['MUNICIPIO']
    if ( mun + " N_RAT " + row['RAT.NUM_ATIVIDADE'] ) in df_classif.index:
        return df_classif.loc[mun+" N_RAT "+row['RAT.NUM_ATIVIDADE'], 'SETOR']    
    elif ( mun + ' BAIRRO ' + row['NOME_BAIRRO'] ) in df_classif.index:
        return df_classif.loc[mun + ' BAIRRO ' + row['NOME_BAIRRO'], 'SETOR']
    elif ( mun + ' LOGRADOURO ' + row['LOGRADOURO'] ) in df_classif.index:
        return df_classif.loc[mun + ' LOGRADOURO ' + row['LOGRADOURO'], 'SETOR']
    elif ( mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'] ) in df_classif.index:
        return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'], 'SETOR']
    elif ( mun + ' COMPLEMENTO_END ' + row['COMPLEMENTO_ENDERECO'] ) in df_classif.index:
        return df_classif.loc[mun + ' COMPLEMENTO_END ' + row['COMPLEMENTO_ENDERECO'], 'SETOR']
    elif ( mun + ' COMPLEMENTO_END ' + row['DES_ENDERECO'] ) in df_classif.index:
        return df_classif.loc[mun + ' COMPLEMENTO_END ' + row['DES_ENDERECO'], 'SETOR']    
    else:
        return 'other'


df_acum['SETOR'] = df_acum.apply(lambda row: classifica_setor(row), axis=1)



# from sqlalchemy import create_engine
# df_acum.to_sql('tbl_rat', 'sqlite:///gdo.db', if_exists='replace')


df_acum[
    ( ~df_acum['MUNICIPIO'].isin([
        'CLAUDIO',
        'CARMO DO CAJURU',
        'ITATIAIUCU',
        'SAO GONCALO DO PARA',
        'ITAUNA'
    ]) ) &
    ( df_acum['SETOR'] == 'other')
][['MUNICIPIO', 'SETOR', 'DTA_INICIO', 'LOGRADOURO', 'DES_ENDERECO',
       'NUM_ENDERECO', 'COMPLEMENTO_ALFA', 'COMPLEMENTO_ENDERECO',
       'NUM_COMPLEMENTAR', 'COD_BAIRRO', 'NOME_BAIRRO', 'TIPO_LOGRADOURO2',
       'LOGRADOURO2', 'DES_ENDERECO2', 'COD_MUNICIPIO', 'MUNICIPIO']]

,MUNICIPIO,SETOR,DTA_INICIO,LOGRADOURO,DES_ENDERECO,NUM_ENDERECO,COMPLEMENTO_ALFA,COMPLEMENTO_ENDERECO,NUM_COMPLEMENTAR,COD_BAIRRO,NOME_BAIRRO,TIPO_LOGRADOURO2,LOGRADOURO2,DES_ENDERECO2,COD_MUNICIPIO,MUNICIPIO
719,DIVINOPOLIS,other,06/12/2018,,CAPELINHA,860,,,,,,,,,312230,DIVINOPOLIS
1613,DIVINOPOLIS,other,26/12/2018,,BRASIL,910,,,,,,,,,312230,DIVINOPOLIS
2527,DIVINOPOLIS,other,30/12/2018,,PARANA,300,,,,,,,,,312230,DIVINOPOLIS
28467,DIVINOPOLIS,other,06/01/2019,BAR ROCINHA,BAR ROCINHA,0,,,,,,,,,312230,DIVINOPOLIS
35783,DIVINOPOLIS,other,08/01/2019,,TUBAROES,500,,,,,,,,,312230,DIVINOPOLIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,DIVINOPOLIS,other,16/01/2020,,VICENTE LUIZ QUADROS,253,,,,,,,,,312230,DIVINOPOLIS
454,DIVINOPOLIS,other,16/01/2020,,CONQUISTA,230,,,,,,,,,312230,DIVINOPOLIS
680,DIVINOPOLIS,other,21/02/2020,,JOAO ANTONIO GONCALVES,251,,,,,,,,,312230,DIVINOPOLIS
13048,DIVINOPOLIS,other,02/04/2020,,ANTONIO FLORENTINO,1045,,,,,,,,,312230,DIVINOPOLIS


In [181]:
df_classif['VALIDADOR_TIPO'].value_counts()

LOGRADOURO                 1009
LOGRADOURO_NAO_CAD          629
BAIRRO_NAO_CAD              593
N_RAT                       476
N_REDS                      475
BAIRRO                      373
COMPLEMENTO_END             124
PONTO_REF                    97
LOGRADOURO_CRUZ               9
LOGRADOURO_CRUZ_NAO_CAD       3
MATO GROSSO                   1
Name: VALIDADOR_TIPO, dtype: int64

In [192]:
df_acum.columns

Index(['RAT.NUM_ATIVIDADE', 'NAT.CODIGO', 'NAT.DESCRICAO', 'DTA_HRA_INCLUSAO',
       'DTA_INICIO', 'HRA_INICIO', 'DTA_TERMINO', 'HRA_TERMINO',
       'DES_ALVO_EVENTO', 'DES_LUGAR', 'NOM_OPERACAO', 'COD_UNIDADE_SERVICO',
       'NOM_UNID_RESPONSAVEL', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'DES_ENDERECO',
       'NUM_ENDERECO', 'COMPLEMENTO_ALFA', 'COMPLEMENTO_ENDERECO',
       'NUM_COMPLEMENTAR', 'COD_BAIRRO', 'NOME_BAIRRO', 'TIPO_LOGRADOURO2',
       'LOGRADOURO2', 'DES_ENDERECO2', 'COD_MUNICIPIO', 'MUNICIPIO',
       'LATITUDE', 'LONGITUDE', 'COD_UNIDADE_AREA', 'NOM_UNIDADE_AREA',
       'DIGITADOR', 'DTA_HRA_INICIO', 'DTA_HRA_INICIO_DT', 'DTA_HRA_TERMINO',
       'DTA_HRA_TERMINO_DT', 'MES', 'ANO', 'TEMPO_DT', 'TEMPO_INT', 'CIA',
       'SETOR'],
      dtype='object')

In [218]:
df_classif.shape

(3789, 11)

In [208]:
df_acum[df_acum['DES_ENDERECO2'] != ""][['NOME_BAIRRO', 'DES_ENDERECO', 'DES_ENDERECO2' ]]

,NOME_BAIRRO,DES_ENDERECO,DES_ENDERECO2
362,SANTA CLARA,SETE DE SETEMBRO,JOAQUIM ANDRE
847,BOM PASTOR,JK,ITAMARANDIBA
883,BOM PASTOR,JK,ITAMARANDIBA
1118,BOM PASTOR,JK,SETE DE SETEMBRO
1226,BOM PASTOR,JK,ITAMARANDIBA
...,...,...,...
42409,CENTRO,PRIMEIRO DE JUNHO,GOIAS
45160,BELA VISTA,MG 260,SANTO ANTONIO
50782,NACOES,BOM SUCESSO,BOLIVIA
52801,PARQUE INDUSTRIAL MARCELINO CORRADI,BANDEIRANTES,JUSCELINO KUBITSCHEK [ JK]


In [193]:
df_classif[
    (df_classif['MUNICIPIO'] == 'DIVINOPOLIS') &
    (df_classif['VALIDADOR_TIPO'] == 'COMPLEMENTO_END')
]

,Unnamed: 0,MUNICIPIO,VALIDADOR,VALIDADOR_TIPO,CIA_BASE,BAIRRO,TIPO,SETOR,SUBSETOR,COD_SETOR,OBSERVACAO
DIVINOPOLIS COMPLEMENTO_END Arena Hangar,48,DIVINOPOLIS,Arena Hangar,COMPLEMENTO_END,53ª CIA PM,CACHOEIRINHA,URBANO,ALTO GOIAS,LP PEREIRA,23.53.2,NaN
DIVINOPOLIS COMPLEMENTO_END BAIRRO LIBERDADE,49,DIVINOPOLIS,BAIRRO LIBERDADE,COMPLEMENTO_END,53ª CIA PM,LIBERDADE,URBANO,BOM PASTOR,PADRE LIBERIO,23.53.3,NaN
DIVINOPOLIS COMPLEMENTO_END CANDIDES,258,DIVINOPOLIS,CANDIDES,COMPLEMENTO_END,142ª CIA PM,JARDIM CANDIDES,URBANO,NITEROI,ICARAI,23.142.2,NaN
DIVINOPOLIS COMPLEMENTO_END del rey,269,DIVINOPOLIS,del rey,COMPLEMENTO_END,142ª CIA PM,DEL REY,URBANO,NITEROI,ICARAI,23.142.2,NaN
DIVINOPOLIS COMPLEMENTO_END dona rosa,510,DIVINOPOLIS,dona rosa,COMPLEMENTO_END,142ª CIA PM,DONA ROSA,URBANO,PORTO VELHO,NACOES,23.142.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
DIVINOPOLIS COMPLEMENTO_END DONA QUITA,3794,DIVINOPOLIS,DONA QUITA,COMPLEMENTO_END,142ª CIA PM,DONA QUITA,URBANO,PORTO VELHO,VALE DO SOL,23.142.1,NaN
DIVINOPOLIS COMPLEMENTO_END BELVEDERE2,3853,DIVINOPOLIS,BELVEDERE2,COMPLEMENTO_END,139ª CIA PM,BELVEDERE,URBANO,SAO JOSE,BELVEDERE,23.139.1,NaN
DIVINOPOLIS COMPLEMENTO_END b:santos dumont,3882,DIVINOPOLIS,b:santos dumont,COMPLEMENTO_END,142ª CIA PM,SANTOS DUMONT,URBANO,PORTO VELHO,VALE DO SOL,23.142.1,NaN
DIVINOPOLIS COMPLEMENTO_END b:icarai,3890,DIVINOPOLIS,b:icarai,COMPLEMENTO_END,142ª CIA PM,ICARAI,URBANO,NITEROI,NaN,23.142.2,NaN


In [172]:
df_acum.columns

Index(['RAT.NUM_ATIVIDADE', 'NAT.CODIGO', 'NAT.DESCRICAO', 'DTA_HRA_INCLUSAO',
       'DTA_INICIO', 'HRA_INICIO', 'DTA_TERMINO', 'HRA_TERMINO',
       'DES_ALVO_EVENTO', 'DES_LUGAR', 'NOM_OPERACAO', 'COD_UNIDADE_SERVICO',
       'NOM_UNID_RESPONSAVEL', 'TIPO_LOGRADOURO', 'LOGRADOURO', 'DES_ENDERECO',
       'NUM_ENDERECO', 'COMPLEMENTO_ALFA', 'COMPLEMENTO_ENDERECO',
       'NUM_COMPLEMENTAR', 'COD_BAIRRO', 'NOME_BAIRRO', 'TIPO_LOGRADOURO2',
       'LOGRADOURO2', 'DES_ENDERECO2', 'COD_MUNICIPIO', 'MUNICIPIO',
       'LATITUDE', 'LONGITUDE', 'COD_UNIDADE_AREA', 'NOM_UNIDADE_AREA',
       'DIGITADOR', 'DTA_HRA_INICIO', 'DTA_HRA_INICIO_DT', 'DTA_HRA_TERMINO',
       'DTA_HRA_TERMINO_DT', 'MES', 'ANO', 'TEMPO_DT', 'TEMPO_INT', 'CIA',
       'SETOR'],
      dtype='object')

In [230]:
df_classif[df_classif['VALIDADOR'] == 'JOAO ANTONIO GONCALVES']

,Unnamed: 0,MUNICIPIO,VALIDADOR,VALIDADOR_TIPO,CIA_BASE,BAIRRO,TIPO,SETOR,SUBSETOR,COD_SETOR,OBSERVACAO


<h2 style="color:blue;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT_VEÍCULOS</h2>
<h3 style="color:red; margin:10px">Importa somente os dados do 23º BPM</h3>

### 1) Fazer descrição

In [18]:
df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfv1 = pd.read_csv('files/RAT/' + rats_viatura_files[0], error_bad_lines=False, sep='|')
# dfv1 = dfv1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv1 = dfv1[dfv1['NUM_ATIVIDADE'].isin(df_rat['RAT.NUM_ATIVIDADE'])]

# dfv2 = pd.read_csv('files/RAT/' + rats_viatura_files[1], error_bad_lines=False, sep='|')
# dfv2 = dfv2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv2 = dfv2[dfv2['NUM_ATIVIDADE'].isin(df_rat['RAT.NUM_ATIVIDADE'])]

# dfv_acum = pd.concat([dfv1, dfv2])


# for rat_viatura_file in rats_viatura_files[2:]:
#     dfv_aux = pd.read_csv('files/RAT/'+rat_viatura_file, sep='|')
#     dfv_aux = dfv_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfv_aux = dfv_aux[dfv_aux['NUM_ATIVIDADE'].isin(df_rat['RAT.NUM_ATIVIDADE'])]
#     dfv_acum = pd.concat([dfv_acum, dfv_aux])
    
# dfv_acum.to_sql('tbl_rat_veiculo', 'sqlite:///gdo.db', if_exists='replace')

# df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
# df_ratv_unique = df_ratv['NUM_ATIVIDADE'].value_counts()
# df_ratv_unique.rename('VEICULOS', inplace=True)
# df_rat_e_v = df_rat.join(df_ratv_unique, how='left')
# df_rat_e_v['VEICULOS'].fillna(0, inplace=True)

# df_rat_e_v.groupby('NOM_UNIDADE_AREA').count()

NameError: name 'df_rat_e_v' is not defined

In [97]:
teste = pd.DataFrame(np.random.randn(5,3), columns=list('ABC'))
teste.to_sql('teste3', 'sqlite:///gdo.db', if_exists='append', index=False)

In [25]:
df_teste = pd.DataFrame({
    'A':[1, 2, 3, 4, 5],
    'B':[3, 7, 9, 13, 17]
})
conds=[
    df_teste['A'] < 3,
    df_teste['B'] % 3 == 0,
    df_teste['A'] < 3
]
results=[
    'X<3',
    'div3',
    'chave'
]

df_teste['C'] = np.select(conds, results, default='other')
df_teste

,A,B,C
0,1,3,X<3
1,2,7,X<3
2,3,9,div3
3,4,13,other
4,5,17,other


In [20]:
df_acum.groupby(['CIA', 'NOM_UNIDADE_AREA']).count()

RAT.NUM_ATIVIDADE  NAT.CODIGO  \
CIA     NOM_UNIDADE_AREA                                                    
0                                                        1688        1688   
139 CIA 139 CIA PM/23 BPM/7 RPM                           962         962   
        3 PEL/139 CIA PM/23 BPM/7 RPM                    3436        3436   
142 CIA 142 CIA PM/23 BPM/7 RPM                          8001        8001   
        2 GP/3 PEL/142 CIA PM/23 BPM/7 RPM               1388        1388   
        3 PEL/142 CIA PM/23 BPM/7 RPM                    4757        4757   
51 CIA  2 GP/3 PEL/51 CIA PM/23 BPM/7 RPM                 187         187   
        51 CIA PM/23 BPM/7 RPM                           4331        4331   
        DESAT5 CIA PM IND/7 RPM                          1260        1260   
        DESATIVADASRACIONAL SEDE/63 BPM                  3964        3964   
        EFETIVO OPERACIONAL SEDE/63 BPM                  1260        1260   
53 CIA  53 CIA PM/23 BPM/7 RPM                          14797       14797   

                                            NAT.DESCRICAO  DTA_HRA_INCLUSAO  \
CIA     NOM_UNIDADE_AREA                                                      
0                                                    1688              1688   
139 CIA 139 CIA PM/23 BPM/7 RPM                       962               962   
        3 PEL/139 CIA PM/23 BPM/7 RPM                3436              3436   
142 CIA 142 CIA PM/23 BPM/7 RPM                      8001              8001   
        2 GP/3 PEL/142 CIA PM/23 BPM/7 RPM           1388              1388   
        3 PEL/142 CIA PM/23 BPM/7 RPM                4757              4757   
51 CIA  2 GP/3 PEL/51 CIA PM/23 BPM/7 RPM             187               187   
        51 CIA PM/23 BPM/7 RPM                       4331              4331   
        DESAT5 CIA PM IND/7 RPM                      1260              1260   
        DESATIVADASRACIONAL SEDE/63 BPM              3964              3964   
        EFETIVO OPERACIONAL SEDE/63 BPM              1260              1260   
53 CIA  53 CIA PM/23 BPM/7 RPM                      14797             14797   

                                            DTA_INICIO  HRA_INICIO  \
CIA     NOM_UNIDADE_AREA                                             
0                                                 1688        1688   
139 CIA 139 CIA PM/23 BPM/7 RPM                    962         962   
        3 PEL/139 CIA PM/23 BPM/7 RPM             3436        3436   
142 CIA 142 CIA PM/23 BPM/7 RPM                   8001        8001   
        2 GP/3 PEL/142 CIA PM/23 BPM/7 RPM        1388        1388   
        3 PEL/142 CIA PM/23 BPM/7 RPM             4757        4757   
51 CIA  2 GP/3 PEL/51 CIA PM/23 BPM/7 RPM          187         187   
        51 CIA PM/23 BPM/7 RPM                    4331        4331   
        DESAT5 CIA PM IND/7 RPM                   1260        1260   
        DESATIVADASRACIONAL SEDE/63 BPM           3964        3964   
        EFETIVO OPERACIONAL SEDE/63 BPM           1260        1260   
53 CIA  53 CIA PM/23 BPM/7 RPM                   14797       14797   

                                            DTA_TERMINO  HRA_TERMINO  \
CIA     NOM_UNIDADE_AREA                                               
0                                                  1688         1688   
139 CIA 139 CIA PM/23 BPM/7 RPM                     962          962   
        3 PEL/139 CIA PM/23 BPM/7 RPM              3436         3436   
142 CIA 142 CIA PM/23 BPM/7 RPM                    8001         8001   
        2 GP/3 PEL/142 CIA PM/23 BPM/7 RPM         1388         1388   
        3 PEL/142 CIA PM/23 BPM/7 RPM              4757         4757   
51 CIA  2 GP/3 PEL/51 CIA PM/23 BPM/7 RPM           187          187   
        51 CIA PM/23 BPM/7 RPM                     4331         4331   
        DESAT5 CIA PM IND/7 RPM                    1260         1260   
        DESATIVADASRACIONAL SEDE/63 BPM            3964         3964   
        EFETI

In [15]:
df_acum[
    (~df_acum['CIA'].isin(['53 CIA', '142 CIA', '139 CIA', '51 CIA']) ) &
    (df_acum['NOM_UNIDADE_AREA'] != "")
][['MUNICIPIO', 'NOM_UNIDADE_AREA', 'CIA']]

,MUNICIPIO,NOM_UNIDADE_AREA,CIA
905,ITAUNA,DESAT5 CIA PM IND/7 RPM,0
1653,ITAUNA,DESAT5 CIA PM IND/7 RPM,0
1841,ITAUNA,DESAT5 CIA PM IND/7 RPM,0
1897,ITAUNA,DESAT5 CIA PM IND/7 RPM,0
1900,ITAUNA,DESAT5 CIA PM IND/7 RPM,0
...,...,...,...
57832,ITAUNA,DESATIVADASRACIONAL SEDE/63 BPM,0
57895,ITAUNA,DESATIVADASRACIONAL SEDE/63 BPM,0
58595,ITATIAIUCU,DESATIVADASRACIONAL SEDE/63 BPM,0
58649,ITATIAIUCU,DESATIVADASRACIONAL SEDE/63 BPM,0


In [12]:
teste = pd.DataFrame(np.random.randn(6, 4), columns=list('WXYZ'))
teste

,W,X,Y,Z
0,-0.203627,1.736455,0.560876,-0.380186
1,0.994563,-1.285162,0.027405,-1.254724
2,0.024210,-0.175574,1.576495,-0.296455
3,-0.732013,0.969298,1.450590,-0.738895
4,0.421194,0.707626,-2.130880,1.058684
5,-0.436871,0.154043,-0.531565,-1.150669


In [30]:
teste.drop(teste[teste['X'] < 0].index, inplace=True)
teste

,W,X,Y,Z
0,-0.203627,1.736455,0.560876,-0.380186
3,-0.732013,0.969298,1.450590,-0.738895
4,0.421194,0.707626,-2.130880,1.058684
5,-0.436871,0.154043,-0.531565,-1.150669
